In [1]:
import glob
import os
import pyfits
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import pywcs
import os, sys
from astropy.io import fits
import matplotlib
%matplotlib inline
from scipy.integrate import quad
import scipy
from scipy import stats
sys.path.append("./reduction/threedhst")
sys.path.append("./reduction/my_python")
sys.path.append("./reduction/aXe2html-1.1")
os.environ["HOME"] = "C:\\cygwin\\home\\rohan naidu"
import threedhst
import threedhst.eazyPy as eazy
from scipy import ndimage
import astropy.io.ascii as ascii
import shutil
import glob


=  THREEDHST.__init__.defaultOptions  =

Initializing THREEDHST parameters

=========== 2015/9/21 8:13 ============


 but the reduction scripts should be OK.



In [2]:
#line catalogue for GOODS-N
north_lines = fits.open("goodsn.linefit.linematched_all.v4.1.5.fits")[1].data

In [3]:
phot_eazy = eazy.TemplateInterpolator(None, MAIN_OUTPUT_FILE='full', OUTPUT_DIRECTORY='./templates/FULL_EAZY/OUTPUT')

In [18]:
file_list = glob.glob('./unicorn/*')

In [5]:
#some string-formatting functions
def dechar(a):
    return (a.split('\\')[1]).split('.')[0]

def get_name(a):
    return a.split('.')[0]

def star_this(element):
    return '*'+element+'*'

star_this_list = np.vectorize(star_this)

dechar_array = np.vectorize(dechar)

get_name_array = np.vectorize(get_name)

#len(np.unique(dechar_array(file_list)))

In [21]:
np.unique(dechar_array(file_list))

array(['goodsn-11-G141-big_07693', 'goodsn-11-G141_07693',
       'goodsn-114-G141-big_24380', 'goodsn-114-G141_24380',
       'goodsn-12-G141-big_18504', 'goodsn-12-G141_18504',
       'goodsn-123-G141-big_12133', 'goodsn-123-G141_12133',
       'goodsn-13-G141-big_18504', 'goodsn-13-G141_18504',
       'goodsn-14-G141-big_20801', 'goodsn-14-G141-big_24380',
       'goodsn-14-G141_20801', 'goodsn-14-G141_24380',
       'goodsn-15-G141-big_25004', 'goodsn-15-G141-big_30053',
       'goodsn-15-G141_25004', 'goodsn-15-G141_30053',
       'goodsn-17-G141-big_34721', 'goodsn-17-G141-big_35204',
       'goodsn-17-G141-big_36765', 'goodsn-17-G141_34721',
       'goodsn-17-G141_35204', 'goodsn-17-G141_36765',
       'goodsn-18-G141-big_36921', 'goodsn-18-G141_36921',
       'goodsn-21-G141-big_07693', 'goodsn-21-G141_07693',
       'goodsn-22-G141-big_11839', 'goodsn-22-G141_11839',
       'goodsn-23-G141-big_11839', 'goodsn-23-G141-big_12133',
       'goodsn-23-G141_11839', 'goodsn-23-G141_1

In [6]:
#this calculates the eqw given an object ID, which line we're looking for(element=OIII, Ha etc), the interval (100A is what Iva suggested) around the target line
#and finally, the degree of the best-fit curve--I've stuck with 1.
def calculate_eqw(id, element, interval_for_continuum, best_fit_poly_degree):
    if element == 'Ha':
        target_wavelength = 6562.81
    elif element == 'OIII':
        target_wavelength = 5000 
#from Iva's grism paper:        
    field = id.split('-')[0]
    nn = int(id.split('_')[1])

    if field.startswith('good'):
            uf = field.replace('goods','goods-').upper()
    else:
            uf = field.upper()

    OUTPUT_DIRECTORY='./unicorn/outdir/'
    MAIN_OUTPUT_FILE = '{}_3dhst.v4.1'.format(field)
    
    new_zfit = fits.open('./unicorn/{}.new_zfit.fits'.format(id))
    coeffs = new_zfit[5].data
    fit = np.dot(phot_eazy.temp_seds, coeffs)
    tmp_temp = np.copy(phot_eazy.temp_seds)
    tmp_temp[:,7] *= 0.
    tmp_temp[:,8] *= 0.
    fit_no_lines = np.dot(tmp_temp, coeffs)
    
    zout = ascii.read(OUTPUT_DIRECTORY+'/'+MAIN_OUTPUT_FILE+'.zout')
    idx = np.where(zout['id'] == nn)[0][0]
    lambdaz, temp_sed, lci, obs_sed, fobs, efobs = eazy.getEazySED(idx, MAIN_OUTPUT_FILE=MAIN_OUTPUT_FILE, OUTPUT_DIRECTORY=OUTPUT_DIRECTORY)

    line_filename = '{}.'
    with open('./unicorn/{}.linefit.dat'.format(id), 'r') as fp:            
            for ll in fp:
                if ll.startswith('# z'):
                    z = float(ll.split('=')[1])
                if ll.startswith('# [xxx]'):
                    values = ll.split()
                    s0, s0_err, s1, s1_err = float(values[-10]), float(values[-8]), \
                        float(values[-4]), float(values[-2])

    fit_no_lines /= (1+z)**2
#x_axis and y_axis are the points in the interval we're considering to get a best-fit line for the continuum level    
    x_axis = (phot_eazy.templam[(phot_eazy.templam*(1+z) < (((z+1)*target_wavelength) + interval_for_continuum)) & (phot_eazy.templam*(1+z) > (((z+1)*target_wavelength) - interval_for_continuum))])*(1+z)
    y_axis = (fit_no_lines[(phot_eazy.templam*(1+z) < (((z+1)*target_wavelength) + interval_for_continuum)) & (phot_eazy.templam*(1+z) > (((z+1)*target_wavelength) - interval_for_continuum))])
#best_fit_lines uses a least squares fit metho
    best_fit_line = np.polyfit(x_axis, y_axis, best_fit_poly_degree)
    best_fit_eqn = np.poly1d(best_fit_line)
#we integrate over the specified interval under the best_fit line. We divide the line flux from the catalogue by the average continnum level for the interval.
    ans, err = quad(best_fit_eqn, (((z+1)*target_wavelength) - interval_for_continuum), (((z+1)*target_wavelength) + interval_for_continuum))

    return ((north_lines[element+'_flux'][nn-1])/(ans/(2*interval_for_continuum)))

In [7]:
#script to test the EQW calculation. It compares our calculated EQW against the EQW from the catalogue.
#the resulting tuple is as follows: (id, calculated EQW, EQW from catalogue, % difference)
def test(element, list_of_ids):
    return np.array([(tu, calculate_eqw(tu,element, 100, 1), north_lines[element+'_EQW'][int(tu.split('_')[1]) - 1], 100*((calculate_eqw(tu,element, 100, 1)-(north_lines[element+'_EQW'][int(tu.split('_')[1]) - 1]))/(north_lines[element+'_EQW'][int(tu.split('_')[1]) - 1]))) for tu in list_of_ids])

In [9]:
def object_list():
    gui_markings_open = fits.open('./unicorn/raw/inspect_3dhst_rpn.info.fits', memmap=True)
    gui_markings= gui_markings_open[1].data
    os.chdir('./unicorn/raw')
    src_files = star_this_list(get_name_array(np.array(gui_markings['images'][np.asarray(gui_markings['sed'], dtype=bool)])))
    for file_name in src_files:
        for element in glob.glob(r'%s'%file_name):
             shutil.copy(element, '..')
    os.chdir('../..')
    gui_markings_open.close()
    return get_name_array(np.array(gui_markings['images'][np.asarray(gui_markings['sed'], dtype=bool)]))

In [104]:
test("Ha", object_list()) #hmag <20 and Halpha>100 minus funky looking objects

array([['goodsn-21-G141_06666', '212.561269315', '217.17', '-2.12217649077'],
       ['goodsn-42-G141_02122', '206.997869783', '212.85', '-2.74941518295']], 
      dtype='|S20')

In [150]:
object_list_output = np.array(object_list())
test("Ha", object_list_output)  #hmag <21 and Halpha>100 minus funky stuff

array([['goodsn-18-G141_37623', '481.591889789', '439.0', '9.70202500893'],
       ['goodsn-18-G141_37738', '179.525890696', '213.07', '-15.7432342909'],
       ['goodsn-21-G141_04711', '155.710912907', '171.99', '-9.46513581794'],
       ['goodsn-21-G141_05487', '213.480624918', '208.81', '2.23678220283'],
       ['goodsn-21-G141_05684', '108.813135963', '110.96', '-1.93480897377'],
       ['goodsn-24-G141_17829', '415.159631533', '384.36', '8.01322498007']], 
      dtype='|S20')

In [10]:
object_list_output = np.array(object_list())
test("Ha", object_list_output) #hmag<22 and Halpha>100

array([['goodsn-12-G141_17432', '191.145716657', '195.32', '-2.13715100497'],
       ['goodsn-13-G141_19258', '273.919352347', '309.48', '-11.490450967'],
       ['goodsn-28-G141_36108', '244.732220987', '270.34', '-9.47243434682'],
       ['goodsn-36-G141_24582', '335.832949096', '383.59', '-12.4500250018'],
       ['goodsn-45-G141_11389', '186.630359559', '192.69', '-3.1447612439'],
       ['goodsn-45-G141_15768', '256.652779806', '304.6', '-15.7410440559']], 
      dtype='|S20')

In [11]:
object_list_output = np.array(object_list())
test("Ha", object_list_output) #hmag<22 and Halpha>100 not as clean as above

array([['goodsn-11-G141_11135', '223.581490119', '222.16', '0.639849711689'],
       ['goodsn-11-G141_15236', '245.545056042', '262.02', '-6.28766657449'],
       ['goodsn-114-G141_20549', '95.0183179649', '113.2', '-16.0615565681'],
       ['goodsn-12-G141_14451', '328.632733188', '382.62', '-14.1098914881'],
       ['goodsn-12-G141_15300', '447.918657093', '486.2', '-7.87357937197'],
       ['goodsn-12-G141_17432', '191.145716657', '195.32', '-2.13715100497'],
       ['goodsn-13-G141_16499', '154.350371901', '176.79', '-12.6928152602'],
       ['goodsn-13-G141_19258', '273.919352347', '309.48', '-11.490450967'],
       ['goodsn-13-G141_22790', '144.082731639', '179.55', '-19.753421532'],
       ['goodsn-14-G141_20549', '95.7607440611', '113.2', '-15.4057031262'],
       ['goodsn-17-G141_35795', '116.582953053', '125.11', '-6.81563979424'],
       ['goodsn-17-G141_37307', '397.72842098', '415.07', '-4.1779890186'],
       ['goodsn-21-G141_04523', '183.872511367', '219.65', '-16.288408